In [1]:
from src import genetic_algorithm
from src.repository.cross_over import SinglePointCO
from src.repository.mutation import  PolynomialM
from src.repository.selection import BinaryTS
from src.repository.population import BinaryPopulation
from src.repository.cross_over import SimulatedBC
from src.repository.population import RealP
from src.repository.selection import RandomSelection
from src.repository.encoding import RealEncoding
from src.repository.penalty import AveragePenalty
from Problems import g1,g4,g5,g6
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def plot_dataframe(df):
    # Extract the "generation" column for the x-axis
    x = df["generation"]

    # Get the list of columns excluding "generation"
    columns_to_plot = [col for col in df.columns if col != "generation"]

    # Plot each column against the "generation" column
    for col in columns_to_plot:
        y = df[col]
        plt.figure()
        plt.plot(x, y)
        plt.title(f"{col} vs. Generation")
        plt.xlabel("Generation")
        plt.ylabel(col)

    # Display the plots
    plt.show()


In [3]:
g1_bound = [[0,1]]*9
g1_bound.extend([[0, 100],[0,100],[0,100], [0, 1]])

g4_bound = [[78,102],[33,45]]
g4_bound.extend([[27,45]]*3)

g5_bound = [[0,1200],[0,1200],[-0.55,0.55],[-0.55,0.55]]
g6_bound = [[13,100],[0,100]]

g1_encoding = RealEncoding(bounds=g1_bound)
g4_encoding = RealEncoding(bounds=g4_bound)
g5_encoding = RealEncoding(bounds=g5_bound)
g6_encoding = RealEncoding(bounds=g6_bound)

In [5]:
def penalty(vilolation):
    return vilolation**2

In [4]:
class G1Penalty(genetic_algorithm.PenaltyInterface):

    def __init__(self):
        self.penalty_factor = 10000
        self.constraints = [
            {"func": lambda x: 2 * x[0] + 2 * x[1] + x[9] + x[10], "lower_bound": None, "upper_bound": 10},
            {"func": lambda x: 2 * x[0] + 2 * x[2] + x[9] + x[11], "lower_bound": None, "upper_bound": 10},
            {"func": lambda x: 2 * x[1] + 2 * x[2] + x[10] + x[11], "lower_bound": None, "upper_bound": 10},
            {"func": lambda x: -8 * x[0] + x[9], "lower_bound": None, "upper_bound": 0},
            {"func": lambda x: -8 * x[1] + x[10], "lower_bound": None, "upper_bound": 0},
            {"func": lambda x: -8 * x[2] + x[11], "lower_bound": None, "upper_bound": 0},
            {"func": lambda x: -2 * x[3] - x[4] + x[9], "lower_bound": None, "upper_bound": 0},
            {"func": lambda x: -2 * x[5] - x[6] + x[10], "lower_bound": None, "upper_bound": 0},
            {"func": lambda x: -2 * x[7] - x[8] + x[11], "lower_bound": None, "upper_bound": 0}
        ]

    def penalty_function(self,violation):
        return penalty(violation)

    


g1_penalty = G1Penalty()

In [6]:
class G4Penalty(genetic_algorithm.PenaltyInterface):

    def __init__(self):
        self.penalty_factor = 10000
        self.constraints = [
            {
                "func": lambda x: 85.334407 + 0.0056858*x[1]*x[4] + 0.00026*x[0]*x[3] - 0.0022053*x[2]*x[4],
                "lower_bound": None,  
                "upper_bound": 92  
            },
            {
                "func": lambda x: 80.51249 + 0.0071217*x[1]*x[4] + 0.0029955*x[0]*x[1] + 0.0021813*x[2]**2,
                "lower_bound": 90,  
                "upper_bound": 110  
            },
            {
                "func": lambda x: 9.300961 + 0.0047026*x[2]*x[4] + 0.0012547*x[0]*x[2] + 0.0019085*x[2]*x[3],
                "lower_bound": 20,  
                "upper_bound": 25  
            },
        ]



    def penalty_function(self,violation):
        return penalty(violation)


g4_penalty = G4Penalty()

In [7]:
class G5Penalty(genetic_algorithm.PenaltyInterface):

    def __init__(self):
        self.penalty_factor = 10000
        self.constraints = [
            {
                "func": lambda x: x[3]-x[2] + 0.55,
                "lower_bound": 0,
                "upper_bound": None
            },
            {
                "func": lambda x: x[2]-x[3] + 0.55,
                "lower_bound": 0,
                "upper_bound": None
            },
            {
                "func": lambda x: 1000 * math.sin(-x[2] - 0.25) + 1000 * math.sin(-x[3] - 0.25) + 894.8 - x[0],
                "lower_bound": 0,
                "upper_bound": None
            },
            {
                "func": lambda x: -(1000 * math.sin(-x[2] - 0.25) + 1000 * math.sin(-x[3] - 0.25) + 894.8 - x[0]),
                "lower_bound": 0,
                "upper_bound": None
            },
            {
                "func": lambda x: 1000 * math.sin(x[2] - 0.25) + 1000 * math.sin(x[2] - x[3] - 0.25) + 894.8 - x[1],
                "lower_bound": 0,
                "upper_bound": None
            },
            {
                "func": lambda x: -(1000 * math.sin(x[2] - 0.25) + 1000 * math.sin(x[2] - x[3] - 0.25) + 894.8 - x[1]),
                "lower_bound": 0,
                "upper_bound": None
            },
            {
                "func": lambda x: 1000 * math.sin(x[3] - 0.25) + 1000 * math.sin(x[3] - x[2] - 0.25) + 1294.8,
                "lower_bound": 0,
                "upper_bound": None
            },
            {
                "func": lambda x: -(1000 * math.sin(x[3] - 0.25) + 1000 * math.sin(x[3] - x[2] - 0.25) + 1294.8),
                "lower_bound": 0,
                "upper_bound": None
            },
        ]



    def penalty_function(self,violation):    
        return penalty(violation)

g5_penalty = G5Penalty()

In [8]:
class G6Penalty(genetic_algorithm.PenaltyInterface):

    def __init__(self):
        self.penalty_factor = 10000
        self.constraints = [
            {
                "func": lambda x: (x[0] - 5)**2 + (x[1] - 5)**2 - 100,
                "lower_bound": 13,  
                "upper_bound": 100  
            },
            {
                "func": lambda x: -(x[0] - 6)**2 - (x[1] - 5)**2 + 82.81,
                "lower_bound": 0,  
                "upper_bound": 100  
            },
        ]


    def penalty_function(self,violation):
        return penalty(violation)

g6_penalty = G6Penalty()

In [9]:
def print_report(report):
    # Iterate over the rows in the report DataFrame
    for index, row in report.iterrows():
        metric = row['Metric']
        with_penalty = row['With Penalty']
        without_penalty = row['Without Penalty']
        
        # Print the results in a structured format
        print(f"{metric}:\n")
        
        if metric == 'Best Solution':
            print("  - With Penalty:")
            print(f"    - Score: {with_penalty['best_eval']:.6f}")
            print(f"    - Feasible: {with_penalty['feasible']}\n")
            print(f"    - Solution: {with_penalty['best_solution']}\n")

            
            print("  - Without Penalty:")
            print(f"    - Score: {without_penalty['best_eval']:.6f}")
            print(f"    - Feasible: {without_penalty['feasible']}\n")
            print(f"    - Solution: {with_penalty['best_solution_no_penalty']}\n")
        else:
            print(f"  - With Penalty: {with_penalty}")
            print(f"  - Without Penalty: {without_penalty}\n")


def generate_report(df):
    # Calculate standard deviation for scores with and without penalty
    std_dev_with_penalty = df['best_eval'].std()
    std_dev_without_penalty = df['best_eval_no_penalty'].std()

    # Find the best solution (minimum score) with and without penalty
    best_solution_with_penalty = df.loc[df['best_eval'].idxmin()]
    best_solution_without_penalty = df.loc[df['best_eval_no_penalty'].idxmin()]


    # Count the number of non-feasible solutions with and without penalty
    non_feasible_with_penalty = len(df[df['feasible'] == False])
    non_feasible_without_penalty = len(df[df['feasible_no_penalty'] == False])

    # Create a report DataFrame with variable column names
    report_data = {
        'Metric': ['Standard Deviation', 'Best Solution', 'Non-Feasible Solutions'],
        'With Penalty': [std_dev_with_penalty, best_solution_with_penalty, non_feasible_with_penalty],
        'Without Penalty': [std_dev_without_penalty, best_solution_without_penalty, non_feasible_without_penalty]
    }
    report_df = pd.DataFrame(report_data)

    return report_df


    


In [10]:
final_report = {}


problems = [('g1',g1,g1_encoding,g1_penalty),('g4',g4,g4_encoding,g4_penalty),('g5',g5,g5_encoding,g5_penalty),('g6',g6,g6_encoding,g6_penalty)]

for problem,_,_,_ in problems:
    final_report[problem] = None

for problem_name,problem,encoding_g,penalty_g in problems:
    print(problem_name)
    best_evaluations = []
    best_eval_no_penalty = []
    feasible = []
    feasible_no_penalty = []
    best_solution = []
    best_solution_no = []
    mutation = PolynomialM(r_mut=0.2)
    cross_over = SimulatedBC()
    population = RealP()
    selection = RandomSelection()
    config = genetic_algorithm.GAConfig(150,150)
    GA_no_penalty = genetic_algorithm.GeneticAlgorithm(cross_over=cross_over,mutation=mutation,population=population,encoding=encoding_g,selection=selection,config=config)
    GA = genetic_algorithm.GeneticAlgorithm(cross_over=cross_over,mutation=mutation,population=population,encoding=encoding_g,selection=selection,config=config,penalty=penalty_g)

    for i in range(30):
        best,best_eval = GA.run(problem,verbose=False)
        best_no,best_eval_no = GA_no_penalty.run(problem,verbose=False)
        best_eval_no_penalty.append(best_eval_no)
        best_evaluations.append(best_eval)
        feasible.append(penalty_g.is_feasible(best))
        feasible_no_penalty.append(penalty_g.is_feasible(best_no))
        best_solution.append(best)
        best_solution_no.append(best_no)

    df = {'best_eval':best_evaluations,
          'feasible':feasible,
          'best_solution':best_solution,
          'best_solution_no_penalty':best_solution_no,
          'best_eval_no_penalty':best_eval_no_penalty,
          'feasible_no_penalty':feasible_no_penalty}

    
    final_report[problem_name] = pd.DataFrame(df)
        


g1


g4


In [ ]:
final_report['g6'].head()

,best_eval,feasible,best_solution,best_solution_no_penalty,best_eval_no_penalty,feasible_no_penalty
0,1.461290e+06,False,"[12.840823081264345, 11.882420359782053]","[11.011909801811811, 7.729441492674571]",-1846.500193,False
1,5.394761e+05,False,"[15.399605193648119, 4.8018743245369]","[17.086417614594197, 2.7547810351585045]",-4772.825475,False
2,5.608677e+05,False,"[15.349702729201997, 3.7767642292550936]","[22.49039310644811, -12.83061381895513]",-33437.825734,False
3,5.388923e+05,False,"[15.39416721338696, 4.976952843282308]","[9.572548759090903, -5.083941350978213]",-15782.997187,False
4,5.389115e+05,False,"[15.39595464808551, 4.920553821128381]","[10.047710233141999, -4.708074115241201]",-15084.005534,False


In [ ]:
for problem_name,problem ,_,_ in problems:
    print('Problem: ', problem_name)
    report = generate_report(final_report[problem_name])
    print_report(report)
    print('\n\n')

Problem:  g6
Standard Deviation:

  - With Penalty: 257792.80631131947
  - Without Penalty: 16900.328245329092

Best Solution:

  - With Penalty:
    - Score: 538881.953179
    - Feasible: False

    - Solution: [15.39523088  4.95949089]

  - Without Penalty:
    - Score: 550791.443126
    - Feasible: False

    - Solution: [22.65450345 -4.38486522]

Non-Feasible Solutions:

  - With Penalty: 30
  - Without Penalty: 30




